# loading data directories

In [1]:
# import yaml
# import os

# def load_config(config_path='/home/boukhari/projects/dental_image_project/configs/config.yaml'):
#     with open(config_path, 'r') as file:
#         config = yaml.safe_load(file)
#     return config

# # Load the configuration
# config = load_config()


In [2]:
# data_dir = config['paths']['data_dir']
# xray_dir = config['paths']['xray_dir']
# photo_dir = config['paths']['photo_dir']

# os.makedirs(xray_dir, exist_ok=True)
# os.makedirs(photo_dir, exist_ok=True)


In [43]:
# data_dir = "/gstock/phenodent/Full images JPEG_AI"
# xray_dir = "/gstock/phenodent/Full images JPEG_AI/xray"
# photo_dir = "/gstock/phenodent/Full images JPEG_AI/photo"

# data_dir = "/gstock/phenodent/Cohort_control_photos"
# xray_dir = "/gstock/phenodent/Cohort_control_photos/xray"
# photo_dir = "/gstock/phenodent/Cohort_control_photos/photo"

# data_dir = "/gstock/phenodent/Cohort_environment_photos"
# xray_dir = "/gstock/phenodent/Cohort_environment_photos/xray"
# photo_dir = "/gstock/phenodent/Cohort_environment_photos/photo"

data_dir = "/gstock/phenodent/DI_anonymised/all"
xray_dir = "/gstock/phenodent/DI_anonymised/all/xray"
photo_dir = "/gstock/phenodent/DI_anonymised/all/photo"

          

In [44]:
import numpy as np
#import cv2
import shutil

# Classifying the images into Photos & Xray
- all images were mixed in the same folder wihtout indication which is photo and which is an xray 
- VGG16 was used to extract features and KNN to distinguish the colored images (Photos) from the greyscale ones (xray)
- manual verification was performed afterward to see if there arent any mistakes (the relatively small number of images and difference in nature (black and white vs colored) allows to skim through them very quickly)

In [45]:
import os
import torch
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, img_path

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load images
#data_dir = config['paths']['data_dir']
image_paths = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith(('.JPG', '.jpg', '.jpeg', '.JPEG', '.png', '.PNG'))]

dataset = CustomDataset(image_paths, transform=transform)
data_loader = DataLoader(dataset, batch_size=32, shuffle=False)


In [46]:
# Load pretrained VGG-16 model
vgg16 = models.vgg16(pretrained=True)
vgg16.classifier = torch.nn.Sequential(*list(vgg16.classifier.children())[:-1])  # Remove the last layer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg16 = vgg16.to(device)

# Extract features
features = []
image_paths_list = []

with torch.no_grad():
    for images, paths in data_loader:
        images = images.to(device)
        outputs = vgg16(images)
        features.append(outputs.cpu().numpy())
        image_paths_list.extend(paths)

features = np.vstack(features)


/home/boukhari/.conda/envs/dent_img_env/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/boukhari/.conda/envs/dent_img_env/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [47]:
from sklearn.cluster import KMeans

# Perform K-means clustering
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(features)
labels = kmeans.labels_


In [48]:
import shutil

# Define paths for classified images with _vgg suffix
xray_dir_vgg = xray_dir 
photo_dir_vgg = photo_dir 

os.makedirs(xray_dir_vgg, exist_ok=True)
os.makedirs(photo_dir_vgg, exist_ok=True)

# Separate images based on clustering labels
cluster_0 = []
cluster_1 = []

for path, label in zip(image_paths_list, labels):
    if label == 0:
        cluster_0.append(path)
    else:
        cluster_1.append(path)

# Determine which cluster is X-ray and which is photo based on the presence of color
cluster_0_colorfulness = np.mean([np.mean(np.array(Image.open(f))) for f in cluster_0])
cluster_1_colorfulness = np.mean([np.mean(np.array(Image.open(f))) for f in cluster_1])

if cluster_0_colorfulness < cluster_1_colorfulness:
    xray_cluster, photo_cluster = cluster_0, cluster_1
else:
    xray_cluster, photo_cluster = cluster_1, cluster_0

# Copy images to respective directories with _vgg suffix
for src_path in xray_cluster:
    filename = os.path.basename(src_path)
    dst_path = os.path.join(xray_dir_vgg, filename)
    shutil.copy(src_path, dst_path)

for src_path in photo_cluster:
    filename = os.path.basename(src_path)
    dst_path = os.path.join(photo_dir_vgg, filename)
    shutil.copy(src_path, dst_path)


# Displaying the images to check 

In [ ]:
import os

import matplotlib.pyplot as plt
num_cols = 5
# Get the list of image filenames in the xray_dir
xray_images = [filename for filename in os.listdir(photo_dir_vgg) if filename.endswith(('.JPG', '.jpg', '.jpeg', '.JPEG', '.png', '.PNG'))]

# Set the number of rows and columns for the grid
num_rows = len(xray_images) // num_cols + 1

# Create a figure and axes for the grid
fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, 60))

# Flatten the axes array
axes = axes.flatten()

# Loop through the xray images and display them in miniatures
for i, filename in enumerate(xray_images):
    image_path = os.path.join(photo_dir_vgg, filename)
    image = plt.imread(image_path)
    axes[i].imshow(image)
    axes[i].axis('off')

# Adjust the spacing between subplots
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
import os

import matplotlib.pyplot as plt
num_cols = 5
# Get the list of image filenames in the xray_dir
xray_images = [filename for filename in os.listdir(xray_dir_vgg) if filename.endswith(('.JPG', '.jpg', '.jpeg', '.JPEG', '.png', '.PNG'))]

# Set the number of rows and columns for the grid
num_rows = len(xray_images) // num_cols + 1

# Create a figure and axes for the grid
fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, 60))

# Flatten the axes array
axes = axes.flatten()

# Loop through the xray images and display them in miniatures
for i, filename in enumerate(xray_images):
    image_path = os.path.join(xray_dir_vgg, filename)
    image = plt.imread(image_path)
    axes[i].imshow(image)
    axes[i].axis('off')

# Adjust the spacing between subplots
plt.tight_layout()

# Show the plot
plt.show()


# checking if there are any files having the same names 
since they come from different sources there might be multiple unique images witht the same name

In [ ]:
# import os
# from collections import defaultdict

# def find_common_files(directories):
#     # Dictionary to keep track of filenames and their occurrences
#     file_occurrences = defaultdict(int)

#     # Iterate over each directory
#     for directory in directories:
#         # Check if the directory exists
#         if not os.path.isdir(directory):
#             print(f"Directory not found: {directory}")
#             continue
        
#         # Iterate over files in the directory
#         for filename in os.listdir(directory):
#             file_occurrences[filename] += 1

#     # Number of directories to compare
#     num_directories = len(directories)

#     # Find files that appear in all directories
#     common_files = [filename for filename, count in file_occurrences.items() if count == num_directories]

#     return common_files

# # List of directories to check
# directories = ["/gstock/phenodent/Full images JPEG_AI",
#                "/gstock/phenodent/Cohort_control_photos",
#                "/gstock/phenodent/Cohort_environment_photos,
#                "/gstock/phenodent/DI_anonymised/all"
# ]

# common_files = find_common_files(directories)

# if common_files:
#     print("Files with the same name in all directories:")
#     for file in common_files:
#         print(file)
# else:
#     print("No common files found in all directories.")


Files with the same name in all directories:
xray
photo


# Counting Files in Directories

In [51]:
import os

def count_files(directory):
    """Count the number of files in a directory."""
    if not os.path.isdir(directory):
        print(f"Directory not found: {directory}")
        return 0
    return len([f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))])

# Define the directories
folder1 = xray_dir
folder2 = photo_dir
folder3 = data_dir

# Count files in each folder separately
folder1_count = count_files(folder1)
folder2_count = count_files(folder2)
folder3_count = count_files(folder3)

# Count files in the combined folders (folder1 + folder2)
combined_folders_count = folder1_count + folder2_count

print(f"Total number of files in {folder1}: {folder1_count}")
print(f"Total number of files in {folder2}: {folder2_count}")
print(f"Total number of files in {folder3}: {folder3_count}")
print(f"Total number of files in {folder1} combined with {folder2}: {combined_folders_count}")


Total number of files in /gstock/phenodent/DI_anonymised/all/xray: 435
Total number of files in /gstock/phenodent/DI_anonymised/all/photo: 76
Total number of files in /gstock/phenodent/DI_anonymised/all: 511
Total number of files in /gstock/phenodent/DI_anonymised/all/xray combined with /gstock/phenodent/DI_anonymised/all/photo: 511


In [17]:
# import os
# import shutil

# def copy_files_to_folder(source_directories, output_directory):
#     # Ensure the output directory exists
#     if not os.path.exists(output_directory):
#         os.makedirs(output_directory)

#     # Iterate over each source directory
#     for directory in source_directories:
#         if not os.path.isdir(directory):
#             print(f"Directory not found: {directory}")
#             continue
        
#         # Walk through the directory and its subdirectories
#         for root, _, files in os.walk(directory):
#             for file in files:
#                 source_file = os.path.join(root, file)
#                 destination_file = os.path.join(output_directory, file)

#                 # # If a file with the same name exists in the output directory, skip it
#                 if os.path.exists(destination_file):
#                     print(f"File already exists: {destination_file}")
#                     continue
                

#                 # Copy the file
#                 shutil.copy2(source_file, destination_file)
#                 print(f"Copied: {source_file} -> {destination_file}")

# # List of source directories to copy files from
# source_directories = [data_dir]

# # Output directory where all files will be copied
# output_directory =  data_dir + "/ALL"

# # Copy files to the output directory
# copy_files_to_folder(source_directories, output_directory)

# print(f"All files have been copied to: {output_directory}")


Copied: /gstock/phenodent/DI_anonymised/all/PAT_C2_0057/IMG_C2_000313.JPG -> /gstock/phenodent/DI_anonymised/all/ALL/IMG_C2_000313.JPG
Copied: /gstock/phenodent/DI_anonymised/all/PAT_C2_0057/IMG_C2_000314.JPG -> /gstock/phenodent/DI_anonymised/all/ALL/IMG_C2_000314.JPG
Copied: /gstock/phenodent/DI_anonymised/all/PAT_C2_0057/IMG_C2_000315.JPG -> /gstock/phenodent/DI_anonymised/all/ALL/IMG_C2_000315.JPG
Copied: /gstock/phenodent/DI_anonymised/all/PAT_C2_0057/IMG_C2_000316.JPG -> /gstock/phenodent/DI_anonymised/all/ALL/IMG_C2_000316.JPG
Copied: /gstock/phenodent/DI_anonymised/all/PAT_C2_0057/IMG_C2_000317.JPG -> /gstock/phenodent/DI_anonymised/all/ALL/IMG_C2_000317.JPG
Copied: /gstock/phenodent/DI_anonymised/all/PAT_C2_0057/IMG_C2_000318.JPG -> /gstock/phenodent/DI_anonymised/all/ALL/IMG_C2_000318.JPG
Copied: /gstock/phenodent/DI_anonymised/all/PAT_C2_0058/IMG_C2_000319.JPG -> /gstock/phenodent/DI_anonymised/all/ALL/IMG_C2_000319.JPG
Copied: /gstock/phenodent/DI_anonymised/all/PAT_C2_0058

# normalizing image extentions

In [53]:
import os

def normalize_jpeg_extensions(directories):
    # Define the valid JPEG extensions
    jpeg_extensions = ['.jpeg', '.jpg', '.JPG', '.PNG', '.png', '.JPEG']

    # Iterate over each directory
    for directory in directories:
        if not os.path.isdir(directory):
            print(f"Directory not found: {directory}")
            continue
        
        # Iterate over each file in the directory
        for filename in os.listdir(directory):
            # Get the file extension
            file_ext = os.path.splitext(filename)[1].lower()
            
            # Check if the file extension is a valid JPEG extension
            if file_ext in jpeg_extensions:
                # Create the new filename with the normalized extension
                new_filename = os.path.splitext(filename)[0] + '.jpg'
                
                # Get full file paths
                old_file = os.path.join(directory, filename)
                new_file = os.path.join(directory, new_filename)
                
                # Rename the file
                os.rename(old_file, new_file)
                print(f"Renamed: {old_file} -> {new_file}")

# List of directories containing the images
directories = ["/gstock/phenodent/Full images JPEG_AI/xray", 
               "/gstock/phenodent/Full images JPEG_AI/photo",
               "/gstock/phenodent/Cohort_control_photos/xray",
               "/gstock/phenodent/Cohort_control_photos/photo", 
               "/gstock/phenodent/Cohort_environment_photos/xray", 
               "/gstock/phenodent/Cohort_environment_photos/photo",
               "/gstock/phenodent/DI_anonymised/all/xray", 
               "/gstock/phenodent/DI_anonymised/all/photo"]

# Normalize JPEG file extensions in the specified directories
normalize_jpeg_extensions(directories)


Renamed: /gstock/phenodent/Full images JPEG_AI/xray/IMG_000015.jpg -> /gstock/phenodent/Full images JPEG_AI/xray/IMG_000015.jpg
Renamed: /gstock/phenodent/Full images JPEG_AI/xray/IMG_000014.jpg -> /gstock/phenodent/Full images JPEG_AI/xray/IMG_000014.jpg
Renamed: /gstock/phenodent/Full images JPEG_AI/xray/IMG_000016.jpg -> /gstock/phenodent/Full images JPEG_AI/xray/IMG_000016.jpg
Renamed: /gstock/phenodent/Full images JPEG_AI/xray/IMG_000017.jpg -> /gstock/phenodent/Full images JPEG_AI/xray/IMG_000017.jpg
Renamed: /gstock/phenodent/Full images JPEG_AI/xray/IMG_000018.jpg -> /gstock/phenodent/Full images JPEG_AI/xray/IMG_000018.jpg
Renamed: /gstock/phenodent/Full images JPEG_AI/xray/IMG_000019.jpg -> /gstock/phenodent/Full images JPEG_AI/xray/IMG_000019.jpg
Renamed: /gstock/phenodent/Full images JPEG_AI/xray/IMG_000020.jpg -> /gstock/phenodent/Full images JPEG_AI/xray/IMG_000020.jpg
Renamed: /gstock/phenodent/Full images JPEG_AI/xray/IMG_000021.jpg -> /gstock/phenodent/Full images JPEG

# checking the image extentions

In [52]:
import os

def find_unique_extensions(directories):
    # Set to store unique file extensions
    unique_extensions = set()

    # Iterate over each directory
    for directory in directories:
        if not os.path.isdir(directory):
            print(f"Directory not found: {directory}")
            continue
        
        # Iterate over each file in the directory
        for filename in os.listdir(directory):
            # Get the file extension
            file_ext = os.path.splitext(filename)[1]
            if file_ext:
                unique_extensions.add(file_ext)

    return unique_extensions

# List of directories to check
directories = ["/gstock/phenodent/Full images JPEG_AI/xray", 
               "/gstock/phenodent/Full images JPEG_AI/photo",
               "/gstock/phenodent/Cohort_control_photos/xray",
               "/gstock/phenodent/Cohort_control_photos/photo", 
               "/gstock/phenodent/Cohort_environment_photos/xray", 
               "/gstock/phenodent/Cohort_environment_photos/photo",
               "/gstock/phenodent/DI_anonymised/all/xray", 
               "/gstock/phenodent/DI_anonymised/all/photo"]
# Find unique file extensions in the specified directories
unique_extensions = find_unique_extensions(directories)

print("Unique file extensions found:")
for ext in unique_extensions:
    print(ext)


Unique file extensions found:
.JPEG
.jpg
.jpeg
.png
.JPG


# Creating a CSV File for Image labels
containing file name, type, cohort

In [56]:
import os
import csv

def create_csv_from_directories(directories, output_csv, col_folder_name='image_type', col_parent_name='category', col_file_name='file_name'):
    # Define a mapping for specific folder names
    folder_name_mapping = {
        'all': 'DI_cohort',
        'Full images JPEG_AI': 'AI_cohort',
        'Cohort_control_photos': 'Control_cohort',
        'Cohort_environment_photos': 'Environment_cohort'
    }

    # Open the CSV file for writing
    with open(output_csv, mode='w', newline='') as file:
        writer = csv.writer(file)
        # Write the header
        writer.writerow([col_folder_name, col_parent_name, col_file_name])
        
        # Iterate over each directory
        for directory in directories:
            if not os.path.isdir(directory):
                print(f"Directory not found: {directory}")
                continue
            
            # Get the parent folder name and the folder name
            parent_folder_name = os.path.basename(os.path.dirname(directory))
            folder_name = os.path.basename(directory)

            # Apply folder name mapping if applicable
            parent_folder_name = folder_name_mapping.get(parent_folder_name, parent_folder_name)
            
            # Iterate over each file in the directory
            for filename in os.listdir(directory):
                file_path = os.path.join(directory, filename)
                if os.path.isfile(file_path):
                    # Write the row to the CSV file
                    writer.writerow([folder_name, parent_folder_name, filename])

# List of directories to check
directories = [
    "/gstock/phenodent/Full images JPEG_AI/xray", 
    "/gstock/phenodent/Full images JPEG_AI/photo",
    "/gstock/phenodent/Cohort_control_photos/xray",
    "/gstock/phenodent/Cohort_control_photos/photo", 
    "/gstock/phenodent/Cohort_environment_photos/xray", 
    "/gstock/phenodent/Cohort_environment_photos/photo",
    "/gstock/phenodent/DI_anonymised/all/xray", 
    "/gstock/phenodent/DI_anonymised/all/photo"
]


# Output CSV file path
output_csv = '/home/boukhari/projects/dental_image_project/data/processed/dental_images.csv'

# Create CSV from subfolders
create_csv_from_directories(directories, output_csv)

print(f"CSV file created: {output_csv}")


CSV file created: /home/boukhari/projects/dental_image_project/data/processed/dental_images.csv


In [2]:
import os
import shutil

def copy_files_to_folder(source_directories, output_directory):
    # Ensure the output directory exists
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Iterate over each source directory
    for directory in source_directories:
        if not os.path.isdir(directory):
            print(f"Directory not found: {directory}")
            continue
        
        # Walk through the directory and its subdirectories
        for root, _, files in os.walk(directory):
            for file in files:
                source_file = os.path.join(root, file)
                destination_file = os.path.join(output_directory, file)

                # If a file with the same name exists in the output directory, append a number to the filename
                counter = 1
                base, extension = os.path.splitext(file)
                while os.path.exists(destination_file):
                    destination_file = os.path.join(output_directory, f"{base}_{counter}{extension}")
                    counter += 1
                    print(f"File already exists: {destination_file}")

                # Copy the file
                shutil.copy2(source_file, destination_file)
                print(f"Copied: {source_file} -> {destination_file}")

# List of source directories to copy files from
source_directories = [
    "/gstock/phenodent/Full images JPEG_AI/xray", 
    "/gstock/phenodent/Full images JPEG_AI/photo",
    "/gstock/phenodent/Cohort_control_photos/xray",
    "/gstock/phenodent/Cohort_control_photos/photo", 
    "/gstock/phenodent/Cohort_environment_photos/xray", 
    "/gstock/phenodent/Cohort_environment_photos/photo",
    "/gstock/phenodent/DI_anonymised/all/xray", 
    "/gstock/phenodent/DI_anonymised/all/photo"
]

# Output directory where all files will be copied
output_directory = '/gstock/phenodent/all_images'

# Copy files to the output directory
copy_files_to_folder(source_directories, output_directory)

print(f"All files have been copied to: {output_directory}")


Copied: /gstock/phenodent/Full images JPEG_AI/xray/IMG_000013.jpg -> /gstock/phenodent/all_images/IMG_000013.jpg
Copied: /gstock/phenodent/Full images JPEG_AI/xray/IMG_000048.jpg -> /gstock/phenodent/all_images/IMG_000048.jpg
Copied: /gstock/phenodent/Full images JPEG_AI/xray/IMG_000099.jpg -> /gstock/phenodent/all_images/IMG_000099.jpg
Copied: /gstock/phenodent/Full images JPEG_AI/xray/IMG_000122.jpg -> /gstock/phenodent/all_images/IMG_000122.jpg
Copied: /gstock/phenodent/Full images JPEG_AI/xray/IMG_000134.jpg -> /gstock/phenodent/all_images/IMG_000134.jpg
Copied: /gstock/phenodent/Full images JPEG_AI/xray/IMG_000135.jpg -> /gstock/phenodent/all_images/IMG_000135.jpg
Copied: /gstock/phenodent/Full images JPEG_AI/xray/IMG_000164.jpg -> /gstock/phenodent/all_images/IMG_000164.jpg
Copied: /gstock/phenodent/Full images JPEG_AI/xray/IMG_000166.jpg -> /gstock/phenodent/all_images/IMG_000166.jpg
Copied: /gstock/phenodent/Full images JPEG_AI/xray/IMG_000220.jpg -> /gstock/phenodent/all_image

In [3]:
# count number of files in directory
import os
len([f for f in os.listdir('/gstock/phenodent/all_images') if os.path.isfile(os.path.join('/gstock/phenodent/all_images', f))])

2186

# Visualizing Image Counts by type

In [1]:
import pandas as pd
df=pd.read_csv('/home/boukhari/projects/dental_image_project/data/processed/dental_images.csv')

In [8]:
# visualize count of each category 
import plotly.express as px
fig = px.histogram(df, x="category", color="image_type", title="Count of each category")
fig.show()

# calculate number of images in each category and image_type
df.groupby(['category', 'image_type']).size()


category            image_type
AI_cohort           photo         556
                    xray           82
Control_cohort      photo         607
                    xray          202
DI_cohort           photo         435
                    xray           76
Environment_cohort  photo         176
                    xray           52
dtype: int64

# removing invisible character from filenames

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('/home/boukhari/projects/dental_image_project/data/raw/dental_images.csv')

# Remove the [U+200E] character (LRM) and any surrounding whitespace
df['file_name'] = df['file_name'].str.replace('\u200e', '').str.strip()

# Update the value where file_name matches 'IMG_C2_000163.jpg'
df.loc[df['file_name'] == 'IMG_C2_000163.jpg', 'image_type'] = 'xray'

# Save the changes back to the CSV file
df.to_csv('/home/boukhari/projects/dental_image_project/data/raw/dental_images.csv', index=False)

# Check if the update was successful
print(df[df['file_name'] == 'IMG_C2_000163.jpg'])


In [ ]:
import pandas as pd
import os

# Define paths
csv_file_path = '/home/boukhari/projects/dental_image_project/data/raw/dental_images.csv'
image_directory_path = '/home/boukhari/projects/dental_image_project/data/raw/all_images'  # Update this to your actual directory

# Load the CSV file
df = pd.read_csv(csv_file_path)

# Extract the list of image names from the CSV
image_names = df['file_name']  
# Get the list of files in the directory
existing_files = set(os.listdir(image_directory_path))

# Check if each image name in the CSV exists in the directory
missing_files = [image_name for image_name in image_names if image_name not in existing_files]

# Output results
if missing_files:
    print("The following files are missing from the directory:")
    for missing_file in missing_files:
        print(missing_file)
else:
    print("All files listed in the CSV are present in the directory.")

In [ ]:
import os

# Define the directory containing the images
directory_path = '/home/boukhari/projects/dental_image_project/data/raw/all_images'  # Update this to your actual directory

# Define the invisible character to remove
invisible_char = '\u200e'  # Left-to-Right Mark (LRM)

# Iterate over all files in the directory
for filename in os.listdir(directory_path):
    # Check if the filename contains the invisible character
    if invisible_char in filename:
        # Construct the new filename by removing the invisible character
        new_filename = filename.replace(invisible_char, '')
        
        # Construct full file paths
        old_file_path = os.path.join(directory_path, filename)
        new_file_path = os.path.join(directory_path, new_filename)
        
        # Rename the file
        os.rename(old_file_path, new_file_path)
        print(f"Renamed: {filename} -> {new_filename}")

print("All files have been processed.")
